# 네이버 매장 -> 매장에 있는 tag 

문제
 
1. 추천순으로 긁어올지, 최신순으로 긁어올지

In [3]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException
from tqdm import tqdm
import chromedriver_autoinstaller
from bs4 import BeautifulSoup

import time
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
import requests
from openpyxl import Workbook
from bs4 import BeautifulSoup
import pandas as pd
import re

from selenium.webdriver.common.action_chains import ActionChains
import warnings 
warnings.filterwarnings('ignore')
from tqdm import tqdm


In [4]:
# 특수문자 제거 함수
# 정규 표현식으로 이모티콘, 특수문자, 아스키코드 제거
def remove_special_characters(text):
    pattern = r'[^\w\s]|_'
    result = re.sub(pattern, '', text)
    return result

In [5]:
# 데이터 불러오기
original_res_df = pd.read_excel('./data/2-1_naver_매장_기본정보_크롤링.xlsx')
original_res_df = original_res_df.drop(['담당'],axis=1)
original_df = original_res_df.copy()
original_df.head()

,식당이름,업태구분,주소,메뉴,가격,방문자리뷰,블로그리뷰,검색어
0,하영호신촌설렁탕 역삼로점,"곰탕,설렁탕",서울 강남구 역삼로 215 1층,설렁탕,"11,000원",397,13,"역삼로 215, 신촌설렁탕"
1,사보텐 압구정본점,돈가스,서울 강남구 압구정로32길 32 1층,시그니처카츠(히레),"18,500원",555,109,압구정로32길 32 캘리스코 사보텐
2,피오렌티나,이탈리아음식,서울 강남구 논현로 841 제이비 미소 빌딩,디너 코스 1인,"70,000원",90,53,논현로 841 피오렌티나
3,압구정변강쇠떡볶이,떡볶이,서울 강남구 도산대로46길 21 한진로즈힐아파트 101동 상가117호,쌀떡볶이(4줄) 1인분,"4,500원",1583,526,도산대로46길 21 압구정변강쇠떡볶이
4,마포만두 역삼점,만두,서울 강남구 논현로 429,메뉴없음,가격없음,25,3,논현로 429 마포만두


In [6]:
# # 일반인 
# import chromedriver_autoinstaller
# chromedriver_autoinstaller.install()
# driver = webdriver.Chrome()

# 지선
driver = webdriver.Chrome(ChromeDriverManager().install())

In [7]:
# 한번에 몇개씩 크롤링할지 정하는 코드 
start_num = 3700 # 할 차례, 여기를 계속 수정해주세요. 하드코딩 ㅠㅋㅋㅋ 0부터 100개를 했으면 그 다음은 start_num = 100
num = 300  # 몇개씩 크롤링할지(기본은 100개를 추천합니다. -> 30분)
end_num = start_num + num
res_df = original_df[start_num:end_num] # 맨 마지막 크롤링할떄는 end_num을 지우세요. [start_num:] <-이렇게 
res_df = res_df.reset_index(drop=True)
res_df.head(3)

,식당이름,업태구분,주소,메뉴,가격,방문자리뷰,블로그리뷰,검색어
0,볼피노,이탈리아음식,서울 강남구 도산대로45길 10-7 신사동 K빌딩,트러플 아란치니,"19,000원",637,1370,"도산대로45길 10-7, 볼피노"
1,뽕나무쟁이 선릉본점,"족발,보쌈",서울 강남구 역삼로65길 31,모둠족발(중),"42,000원",2462,2229,역삼로65길 31 뽕족본점
2,새마을식당 역삼스타타워점 직화,돼지고기구이,서울 강남구 테헤란로20길 19 엘지역삼에클라트 1층 새마을식당 직화구이전문점,열탄불고기(150g),"10,900원",439,89,테헤란로20길 19 새마을식당


In [8]:
# 빈 리스트 생성
restaurant_name_list = []
category_name_list = []
price_list = []


# 반복해서 태그를 수집하기 
for i in tqdm(range(0, len(res_df)), desc='진행 중', position=0, leave=True):
    name = res_df['검색어'][i]
    driver.get('https://map.naver.com/p/search/{}'.format(name))
    time.sleep(3)  

    try :
        if driver.find_elements(By.ID,'entryIframe') :
            entryIframe = driver.find_element(By.ID,'entryIframe')
            driver.switch_to.frame(entryIframe)
    except :
        pass 

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')


    # 매장 정보에서 '리뷰'를 클릭하기 위해서 리뷰 href 속성을 가져온다. 
    try : 
        # '리뷰' 탭의 href 속성 가져오기
        review_tab = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//a[contains(text(), "리뷰")]'))
        )

        # 리뷰 탭의 a 주소 가져오기
        review_url = review_tab.get_attribute('href')
        if review_url == 'https://m.place.naver.com/my/policy/visitorReview' :
            continue
        else : 
            driver.get(review_url)
            time.sleep(2.2)  
    except :
        continue 


    # # 리뷰 탭으로 이동합니다.      
    # driver.get(review_url)
    # time.sleep(2.2)  

    try :
        # 리뷰 탭에서 '더보기' 버튼을 클릭합니다. 
        more_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'a.dP0sq'))
        )
        more_button.click()
    except :
        pass

    # 끝까지 스크롤을 합니다. 
    for _ in range(4):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1.5) 

    # HTML 추출
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.select('ul.K4J9r li.MHaAm, ul.K4J9r li.MHaAm')


    # 매장명/태그/인원수를 가져옵니다. 
    
    for item in items:
        restaurant_name_list.append(name)
        try :
            category_name_list.append(item.text.split('"')[1])
            price_list.append(int(''.join(filter(str.isdigit, item.text))))
        except :
            category_name_list.append( '리뷰 10개 미만')
            price_list.append('')
     
    # 차곡차곡 저장합니다. 
    data_list = {
        '매장명': restaurant_name_list,
        'tag': category_name_list,
        '인원': price_list }

result_df = pd.DataFrame(data_list)

# 엑셀로 저장합니다. 
result_df.to_excel(f'./data/restaurant_tag_data/restaurant_tag_df_{start_num}_{end_num}.xlsx', index=False)


진행 중: 100%|██████████| 300/300 [1:21:25<00:00, 16.29s/it]


In [1]:
# 폴더 안에 있는 엑셀 파일 하나로 합치기 

import pandas as pd
import os

# 폴더 경로 설정
folder_path = './Data/restaurant_tag_data'

# 폴더 내의 모든 xlsx 파일 목록 가져오기
file_list = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]

# 빈 데이터프레임 리스트 생성
df_list = []

# 모든 xlsx 파일 읽어서 데이터프레임 리스트에 추가
for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)
    xls = pd.ExcelFile(file_path)
    for sheet_name in xls.sheet_names:
        df = pd.read_excel(file_path, sheet_name=sheet_name)
        df_list.append(df)

# 모든 데이터프레임을 하나로 합치기
combined_df = pd.concat(df_list, ignore_index=True)

# 결과를 새로운 xlsx 파일로 저장
output_file = 'combined_restaurant_tags.xlsx'
combined_df.to_excel(output_file, index=False)

print(f'All data has been combined and saved to {output_file}')


ValueError: No objects to concatenate

# ------------------------아래는 리뷰---------------------------------

# 음식점 리뷰 크롤링

In [ ]:
# # 빈 리스트 생성
# user_id_list = []
# content_list = []
# date_list = []  
# revisit_list = []
# restaurant_name_list = []
# sub_info_list = []
# # visit_method_list = []
# # wait_time_list = []
# # purpose_list = []
# # visit_with_list =[]

# # 해민 
# # webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정

# chromedriver_autoinstaller.install()
# driver = webdriver.Chrome()
# # 지선
# # driver = webdriver.Chrome(ChromeDriverManager().install())

# scroll_num = 5
# total_iterations = len(res_df)
# for i in tqdm(range(total_iterations),desc='진행 중',position=0, leave= True):
  
#     name = res_df['검색어'][i]
#     driver.get('https://map.naver.com/p/search/{}'.format(name))

#     time.sleep(3)  
#     try :
#         if driver.find_elements(By.ID,'entryIframe') :
#             entryIframe = driver.find_element(By.ID,'entryIframe')
#             driver.switch_to.frame(entryIframe)
#     except :
#         pass 

#     html = driver.page_source
#     soup = BeautifulSoup(html, 'html.parser')

#     try : 
#         # '리뷰' 탭의 href 속성 가져오기
#         review_tab_href = soup.find('a', {'class': 'tpj9w _tab-menu', 'aria-selected': 'true'}).get('href')
#         review_url = 'https://pcmap.place.naver.com'+review_tab_href

#         driver.get(review_url)
#         time.sleep(2.2)  

#     except :
#         print('끝')

#     # 현재 페이지 URL 가져오기
#     current_url = driver.current_url

#     try : 
#         # 리뷰 url로 이동 
#         modified_url = current_url.replace('/home', '/review/visitor')
#         driver.get(modified_url)
#     except :
#         continue
#         # 리뷰 스크래핑 시작
#     time.sleep(3)
#     try:
#         # ------------ 리뷰, 태그 '내용 더보기' 버튼 클릭 ----------------------
#         review_buttons = driver.find_elements(By.CLASS_NAME,'xHaT3')  
#         for button1 in review_buttons:
#             try:
#                 button1.click()
#             except :
#                 pass
                
            
#             tag_buttons = driver.find_elements(By.CSS_SELECTOR,'.sIv5s.WPk67')  
#             for button2 in tag_buttons:
#                 try:
#                     button2.click()
#                 except :
#                     pass
#          # 리뷰 크롤링 시작
#         html = driver.page_source
#         soup = BeautifulSoup(html, 'lxml')
#         reviews = soup.select('li.owAeM') 
#         for r in reviews:
#             user_id = r.select_one('div.qgLL3 span.P9EZi')
#             content = r.select_one('div.vg7Fp span.zPfVt')
#             sub_info = r.select_one('div.MnhVd a.SbJ__').text if r.select_one('div.MnhVd a.SbJ__') else ' '
#             date = r.select_one('div.jxc2b div.D40bm span.CKUdu time')
#             revisit = r.select_one('div.jxc2b div.D40bm span.CKUdu:nth-of-type(2)')
            
#             # 없는 경우 
#             user_id_text = user_id.text if user_id else ''
#             content_text = content.text if content else ''          
#             date_text = date.text if date else ''
#             revisit_text = revisit.text.replace('번째 방문', '') if revisit else ''
            
#             # 리스트에 합치기
#             restaurant_name_list.append(name)
#             user_id_list.append(user_id_text)
#             sub_info_list.append(sub_info)

#             date_list.append(date_text)
#             revisit_list.append(revisit_text)
#             content_list.append(remove_special_characters(content_text))

#     except Exception as e:
#         print(e)
# # 데이터프레임 얹히기
# review_data_list = {
#     '검색어': restaurant_name_list,
#     'user_id': user_id_list,
#     'review': content_list,
#     'date': date_list,
#     'revisit': revisit_list,
#     'sub_info' : sub_info_list
# }
# driver.quit()
# # 리뷰 데이터 프레임 완성
# review_result_df = pd.DataFrame(review_data_list)

# review_result_df



# ## 결과 데이터에 검색어 칼럼 추가
# ## 추후에 인덱스로 쓸 예정이기 떄문

# review_result_df.to_excel(f'./data/restaurant_review_data/restaurant_review_df_{start_num}_{end_num}.xlsx', index=False)

In [ ]:
        # #스크롤
        # for _ in range(scroll_num):
        #     current_scroll_position = driver.execute_script('return window.scrollY;') # 초기 스크롤 위치 

        #     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        #     time.sleep(2) 
            
        #     # # 'TeItc' 클래스를 가진 요소가 클릭 가능할 때까지 기다리고, 보이도록 스크롤
        #     # button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'TeItc')))
        #     # driver.execute_script("arguments[0].scrollIntoView();", button)
        #     # button.click()
        #     # # 더보기 클릭
        #     # button = driver.find_element(By.CLASS_NAME,'fvwqf')
        #     # button.click()
        #     # # 'fvwqf' 클래스를 가진 요소가 나타날 때까지 스크롤
        #     # driver.execute_script("arguments[0].scrollIntoView();", button)
        #     # button.click()

        #     # ------------ 리뷰, 태그 '내용 더보기' 버튼 클릭 ----------------------
        #     review_buttons = driver.find_elements(By.CLASS_NAME,'xHaT3')  
        #     for button1 in review_buttons:
        #         try:
        #             button1.click()
        #         except :
        #             pass

In [ ]:

# # 리뷰는 최대 11개
# # url도 가져오겠습니다.


# # 빈 리스트 생성
# user_id_list = []
# content_list = []
# date_list = []  
# revisit_list = []
# tag_category_name_list = []
# restaurant_name_list = []
# review_category_name_list = []

# # 해민 
# # webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정

# chromedriver_autoinstaller.install()
# driver = webdriver.Chrome()
# # 지선
# # driver = webdriver.Chrome(ChromeDriverManager().install())

# scroll_num = 5

# for i in range(0,len(new_res_df_test)):
  
#     name = new_res_df_test['검색어'][i]
#     driver.get('https://map.naver.com/p/search/{}'.format(name))

#     time.sleep(3)  
#     try :
#         if driver.find_elements(By.ID,'entryIframe') :
#             entryIframe = driver.find_element(By.ID,'entryIframe')
#             driver.switch_to.frame(entryIframe)
#     except :
#         pass 

#     html = driver.page_source
#     soup = BeautifulSoup(html, 'html.parser')

#     try : 
#         # '리뷰' 탭의 href 속성 가져오기
#         review_tab_href = soup.find('a', {'class': 'tpj9w _tab-menu', 'aria-selected': 'true'}).get('href')
#         review_url = 'https://pcmap.place.naver.com'+review_tab_href

#         driver.get(review_url)
#         time.sleep(2.2)  

#     except :
#         print('끝')

#     # 현재 페이지 URL 가져오기
#     current_url = driver.current_url

#     try : 
#         # 리뷰 url로 이동 
#         modified_url = current_url.replace('/home', '/review/visitor')
#         driver.get(modified_url)
#     except :
#         continue

#     # 리뷰 스크래핑 시작
#     try:
#         # 스크롤
#         for _ in range(scroll_num):
#             current_scroll_position = driver.execute_script('return window.scrollY;') # 초기 스크롤 위치 

#             driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#             time.sleep(2) 
            
#             # 'TeItc' 클래스를 가진 요소가 클릭 가능할 때까지 기다리고, 보이도록 스크롤
#             button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'TeItc')))
#             driver.execute_script("arguments[0].scrollIntoView();", button)
#             button.click()

#             # # 더보기 클릭
#             # button = driver.find_element(By.CLASS_NAME,'fvwqf')
#             # button.click()

#             # # 'fvwqf' 클래스를 가진 요소가 나타날 때까지 스크롤
#             # driver.execute_script("arguments[0].scrollIntoView();", button)
#             # button.click()

#             # ------------ 리뷰, 태그 '내용 더보기' 버튼 클릭 ----------------------
#             # review_buttons = driver.find_elements(By.CLASS_NAME,'xHaT3')  
#             # for button1 in review_buttons:
#             #     try:
#             #         button1.click()
#             #     except :
#             #         pass

#             tag_buttons = driver.find_elements(By.CLASS_NAME,'.gyAGI a.P1zUJ.ZGKcF')  
#             for button2 in tag_buttons:
#                 try:
#                     button2.click()
#                 except :
#                     pass
#             # ----------------------------------------------
#             time.sleep(3)

#             # ---------------스크롤위치가 변하지 않으면 리뷰를 다 크롤링한 것이니까 탈출 -------------------------
#             # 현재 스크롤 
#             new_scroll_position = driver.execute_script('return window.scrollY;')

#             # 현재 위치와 이전 위치가 같으면 반복문 탈출
#             if new_scroll_position == current_scroll_position:
#                 break

#             # 이전 스크롤 위치 업데이트
#             prev_scroll_position = current_scroll_position
#             # -------------------------------------------------------------------------------------------------
            
#             html = driver.page_source
#             soup = BeautifulSoup(html, 'lxml')
#             reviews = soup.select('li.owAeM') 

#         # 리뷰 크롤링 시작
#         for r in reviews:
#             user_id = r.select_one('div.qgLL3 span.P9EZi')
#             review_count = r.select_one('div.Gt2_9 span.RNn6x:nth-of-type(1)')
#             photo_count = r.select_one('div.Gt2_9 span.RNn6x:nth-of-type(2)')
            
#             content = r.select_one('div.vg7Fp span.zPfVt')
            
#             try:
#                 tag = r.select_one('div.ERkm0 span.sIv5s')
#             except:
#                 tag = ""
            
#             date = r.select_one('div.jxc2b div.D40bm span.CKUdu time')
#             revisit = r.select_one('div.jxc2b div.D40bm span.CKUdu:nth-of-type(2)')
            
#             # 없는 경우 
#             user_id_text = user_id.text if user_id else ''
#             review_count_text = review_count.text if review_count else ''
#             photo_count_text = photo_count.text if photo_count else ''
#             content_text = content.text if content else ''
#             tag_text = tag.text if tag else ''
#             date_text = date.text if date else ''
#             revisit_text = revisit.text.replace('번째 방문', '') if revisit else ''
            
#             # 리스트에 합치기
#             restaurant_name_list.append(name)
#             user_id_list.append(user_id_text)
#             review_category_name_list.append(tag_text)
#             date_list.append(date_text)
#             revisit_list.append(revisit_text)
#             content_list.append(remove_special_characters(content_text))

#     except Exception as e:
#         print(e)


# # 데이터프레임 얹히기
# review_data_list = {
#     '검색어': restaurant_name_list,
#     'user_id': user_id_list,
#     'review': content_list,
#     'date': date_list,
#     'revisit': revisit_list,
#     'tag': review_category_name_list
# }

# # 리뷰 데이터 프레임 완성
# review_result_df = pd.DataFrame(review_data_list)

# review_result_df

In [ ]:
# review_result_df.to_excel(f'./data/restaurant_review_data/restaurant_review_df_{start_num}_{end_num}.xlsx', index=False)